In [59]:
from instagram_web_api import Client, ClientCompatPatch, ClientError, ClientLoginError
from datetime import datetime

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import PunktSentenceTokenizer 
from nltk.tokenize import PunktSentenceTokenizer 
from nltk.corpus import webtext 
from nltk.stem.porter import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 

import hashlib
import string
import random
import time
import os
import codecs

# просто Client не работает, поэтому переопределена функция _extract_rhx_gis для захода в web инста
class MyClient(Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode())

def getWebAPI():
    time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
    return MyClient(auto_patch = True, drop_incompat_keys = False)

In [60]:
class Comment:
    
    # структурка для коммента, в которой хранятся id поста, автор, время, текст коммента,
    # также отредактированный текст коммента и его эмоциональный окрас
    
    def __init__(self, media_id, com_author, com_time, com_text, aggregated_com_text = '', emotional_color = 'neu'):
        self.media_id = media_id
        self.com_author = com_author
        self.com_time = com_time
        self.com_text = com_text
        self.aggregated_com_text = aggregated_com_text
        self.emotional_color = emotional_color
    
class Post:
    
    # структурка для поста: id поста и число комментов к нему
    
    def __init__(self, media_id, comments_count):
        self.media_id = media_id
        self.comments_count = comments_count
    
# возвращает id юзера по нику
def getUserID(api, username):
    time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
    return api.user_info2(username)['id']

# возвращает список символов, прочитанных из файла (для считывания смайликов и разрешенных в нике символов)
def loadSymbolsList(rel_path):
    # получаем абссолютный путь до файла
    abs_path = os.path.join(os.getcwd(), os.path.normpath(rel_path))
    symbols_list = []
    
    with codecs.open(abs_path, 'r', encoding='utf-8', errors='ignore') as file_data:
        for line in file_data:
            for symb in line:
                if (symb != '\n' and symb != '\r'):
                    symbols_list.append(symb)
    
    return symbols_list

# возвращает media_id всех медиа профиля
def getProfileMediaList(api, username):
    user_id = getUserID(api, username)
    
    media_list = []
    next_page = ''
    has_next_page = True
    
    while (has_next_page):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        all_media = api.user_feed(user_id, count = 50, end_cursor = next_page, extract = False)
        
        # записываем id полученных медиа профиля в лист
        for item in all_media['data']['user']['edge_owner_to_timeline_media']['edges']:
            media_list.append(Post(item['node']['shortcode'], item['node']['edge_media_preview_comment']['count']))
        
        # смотрим, есть ли следующая страница
        has_next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        
        # присваиваем ссылку на следующую страницу
        next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        
    return media_list

# возвращает список всех комментов к посту
def getPostComments(api, media_id):
    res = []
    max_id = ' '
    next_max_id = ''
    
    while (max_id != next_max_id):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        max_id = next_max_id
        post_comments = api.media_comments(media_id, count = 50, end_cursor = max_id) # загружаем комменты
        
        # если комменты кончились, ливаем
        if (len(post_comments) == 0):
            break
        
        # смотрим, на каком комменте закончилась выгрузка
        # реверсим массив комментов, чтобы итоговый список получился в убывающем порядке
        next_max_id = post_comments[0]['id']
        post_comments.reverse()
        
        # оставляем только нужную информацию
        for comm in post_comments:
            res.append(Comment(media_id, comm['owner']['username'], comm['created_at'], comm['text']))
    
    return res

# возвращает все комменты к списку постов
def getPostsComments(api, posts):
    res = []
    for item in posts:
        res.extend(getPostComments(api, item.media_id))
    
    return res

# возвращает общее число комментариев к списку постов
def getSumCommentsCount(posts):
    cnt = 0
    for item in posts:
        cnt += item.comments_count

    return cnt

# возвращает общее число постов с комментариями из заданного списка постов
def getPostsWithCommentsCount(posts):
    cnt = 0
    for item in posts:
        if (item.comments_count > 0):
            cnt += 1

    return cnt

# производит следующие операции с текстом комментов:
# убирает все эмодзи из коммента
# удаляет лишние пробелы
# заменяет все ссылки на никнейм типа @mr_justadog на нейтральное обращение - man
# записывает новую версию коммента в поле aggregated_com_text
def filterComments(comments, permitted_nickname_symbols, all_emoji_list):
    for i in range(len(comments)):
        l = 0
        r = 0
        comments[i].aggregated_com_text = ''
        last_added_symb = 'n'
        while (l + 1 < len(comments[i].com_text)):
            if (comments[i].com_text[l] == '@'):
                r += 1
                while (r + 1 < len(comments[i].com_text) and comments[i].com_text[r] in permitted_nickname_symbols):
                    r += 1
                # если это все же никнейм, а не просто символ '@'
                if (r - l > 1):
                    comments[i].aggregated_com_text += 'Man'
                    last_added_symb = 'n'
                l = r
            elif (comments[i].com_text[l] not in all_emoji_list):
                comments[i].aggregated_com_text += comments[i].com_text[l]
                last_added_symb = comments[i].com_text[l]
                l += 1
            elif (last_added_symb != ' ' or comments[i].com_text[l] != ' '):
                last_added_symb = comments[i].com_text[l]
                l += 1
            else:
                l += 1
        
# заполняет поле emotional_color в списке comments
def getEmotionalColorComments(comments, positive_emoji_list, negative_emoji_list):
    # загружаем анализатор
    sia = SentimentIntensityAnalyzer()
    
    # относим коммент к одному из 3-х типов
    # запускаем классификатор на аггрегированном комменте
    # если коммент определяется классификатором как нейтральный, смотрим на присутствующие в оригинальном комменте смайлы
    # в таком случае, если присутствуют абсолютно положительные смайлы, то коммент положительный
    # если абсолютно положительных смайлов нет, коммент негативный
    # иначе коммент остается нейтральным
    for i in range(len(comments)):
        scores = sia.polarity_scores(comments[i].aggregated_com_text)
        neg = scores['neg']
        neu = scores['neu']
        pos = scores['pos']
        
        if (pos > neg and pos >= neu):
            comments[i].emotional_color = 'pos'
        elif (neg > pos and neg >= neu):
            comments[i].emotional_color = 'neg'
        else:
            for emoji in negative_emoji_list:
                if (emoji in comments[i].com_text):
                    comments[i].emotional_color = 'neg'
                    break
            for emoji in positive_emoji_list:
                if (emoji in comments[i].com_text):
                    comments[i].emotional_color = 'pos'
                    break

# возвращает число положительных, отрицательных и нейтральных комментов из предоставленного списка комментов
def getPosNegNeuCommentsCount(comments):
    pos = 0
    neg = 0
    for i in range(len(comments)):
        if (comments[i].emotional_color == 'neg'):
            neg += 1
        elif (comments[i].emotional_color == 'pos'):
            pos += 1
    return pos, neg, len(comments) - pos - neg

# возвращает число положительно, отрицательно и нейтрально оцененных постов из предоставленного списка комментов к постам
def getPosNegNeuPostsCount(comments):
    pos = 0
    neg = 0
    neu = 0
    cur_pos = 0
    cur_neg = 0
    cur_neu = 0
    last_media_id = comments[0].media_id if len(comments) > 0 else ''
    for i in range(len(comments)):
        if (last_media_id != comments[i].media_id):
            if (cur_pos > cur_neg):
                pos += 1
            elif (cur_neg > cur_pos):
                neg += 1
            elif (cur_neu + cur_neg + cur_pos > 0):
                neu += 1
            cur_pos = cur_neg = cur_neu = 0
        if (comments[i].emotional_color == 'neg'):
            cur_neg += 1
        elif (comments[i].emotional_color == 'pos'):
            cur_pos += 1
        else:
            cur_neu += 1
        last_media_id = comments[i].media_id
    if (cur_pos > cur_neg):
        pos += 1
    elif (cur_neg > cur_pos):
        neg += 1
    elif (cur_neu + cur_neg + cur_pos > 0):
        neu += 1
    return pos, neg, neu

# возвращает id поста из ссыслки на пост
def getPostIDByLink(link):
    parts = link.split('/')
    return parts[4]

# возвращает число уникальных авторов комментариев из предоставленного списка комментов
def getCommentatorsCount(comments):
    commentators = set()
    for i in range(len(comments)):
        if (comments[i].com_author not in commentators):
            commentators.add(comments[i].com_author)
    
    return len(commentators)

# возвращает статистику профиля
# информация хранится в массиве, где:
# 0 элемент - число постов с комментами
# 1 элемент - число постов без комментов
# 2 элемент - число положительно оцененных постов
# 3 элемент - число негативно оцененных постов
# 4 элемент - число нейтрально оцененных оцененных постов
# 5 элемент - число комментов в профиле
# 6 элемент - число положительных комментариев
# 7 элемент - число негативных комментариев
# 8 элемент - число нейтральных комментариев
# 9 элемент - число различных комментаторов в профиле
def getProfileStats(api, username):
     # получаем список постов в профиле
    posts = getProfileMediaList(api, username)

    # общее число постов с комментами
    posts_with_comments_cnt = getPostsWithCommentsCount(posts)

    # общее число постов без комментов
    posts_without_comments_cnt = len(posts) - posts_with_comments_cnt

    # получаем список всех комментов в профиле
    comments = getPostsComments(api, posts)
    # немного причесываем комменты
    filterComments(comments, permitted_nickname_symbols, all_emoji_list)
    # вычисляем эмоциональный окрас комментов
    getEmotionalColorComments(comments, positive_emoji_list, negative_emoji_list)

    # число положительных, отрицательных и нейтральных постов в профиле
    pos_posts_cnt, neg_posts_cnt, neu_posts_cnt = getPosNegNeuPostsCount(comments)

    # общее число комментов к профилю
    comments_cnt = getSumCommentsCount(posts)

    # число положительных, отрицательных и нейтральных комментов в профиле
    pos_comms_cnt, neg_comms_cnt, neu_comms_cnt = getPosNegNeuCommentsCount(comments)
    
    # число различных комментаторов в профиле
    commentators_cnt = getCommentatorsCount(comments)
    
    # заполняем итоговый массив
    stats = []
    stats.append(posts_with_comments_cnt)
    stats.append(posts_without_comments_cnt)
    stats.append(pos_posts_cnt)
    stats.append(neg_posts_cnt)
    stats.append(neu_posts_cnt)
    stats.append(comments_cnt)
    stats.append(pos_comms_cnt)
    stats.append(neg_comms_cnt)
    stats.append(neu_comms_cnt)
    stats.append(commentators_cnt)
    
    return stats

# возвращает статистику поста
# информация хранится в массиве, где:
# 0 элемент - число положительных комментариев
# 1 элемент - число негативных комментариев
# 2 элемент - число нейтральных комментариев
# 3 элемент - число различных комментаторов к посту
def getPostStats(api, post_link):
    # получаем id поста
    posts = [Post(getPostIDByLink(post_link), 0)]

    # получаем список всех комментов к посту
    comments = getPostsComments(api, posts)
    # немного причесываем комменты
    filterComments(comments, permitted_nickname_symbols, all_emoji_list)
    # вычисляем эмоциональный окрас комментов
    getEmotionalColorComments(comments, positive_emoji_list, negative_emoji_list)

    # число положительных, отрицательных и нейтральных комментов в профиле
    pos_comms_cnt, neg_comms_cnt, neu_comms_cnt = getPosNegNeuCommentsCount(comments)
    
    # число различных комментаторов к посту
    commentators_cnt = getCommentatorsCount(comments)
    
    # заполняем итоговый массив
    stats = []
    stats.append(pos_comms_cnt)
    stats.append(neg_comms_cnt)
    stats.append(neu_comms_cnt)
    stats.append(commentators_cnt)
    
    return stats

def printProfileStats(stats):
    print('posts with comments count: ' + str(stats[0]))
    print('posts without comments count: ' + str(stats[1]))
    print('positive posts count: ' + str(stats[2]))
    print('negative posts count: ' + str(stats[3]))
    print('neutral posts count: ' + str(stats[4]))
    print('comments count: ' + str(stats[5]))
    print('positive comments count: ' + str(stats[6]))
    print('negative comments count: ' + str(stats[7]))
    print('neutral comments count: ' + str(stats[8]))
    print('commentators cnt: ' + str(stats[9]))
    
def printPostStats(stats):
    print('positive comments count: ' + str(stats[0]))
    print('negative comments count: ' + str(stats[1]))
    print('neutral comments count: ' + str(stats[2]))
    print('commentators cnt: ' + str(stats[3]))

In [61]:
api = getWebAPI()
username = 'k1pnis'
post_link = 'https://www.instagram.com/p/CATDOM7JfYR/'
all_emoji_list = loadSymbolsList('helper_files\\all_emoji_list')
positive_emoji_list = loadSymbolsList('helper_files\\positive_emoji_list')
negative_emoji_list = loadSymbolsList('helper_files\\negative_emoji_list')
permitted_nickname_symbols = loadSymbolsList('helper_files\\nickname_symbols_list')

In [63]:
post_stats = getPostStats(api, post_link)
printPostStats(post_stats)

positive comments count: 3461
negative comments count: 36
neutral comments count: 1438
commentators cnt: 3839


In [64]:
profile_stats = getProfileStats(api, username)
printProfileStats(profile_stats)

posts with comments count: 50
posts without comments count: 0
positive posts count: 34
negative posts count: 5
neutral posts count: 11
comments count: 759
positive comments count: 198
negative comments count: 49
neutral comments count: 502
commentators cnt: 132
